In [ ]:
from sqlalchemy import create_engine

# MySQL connection parameters
host= 'localhost'
user= 'josh'
password= 'go$T4GS'
database= 'data_4999'

# Create MySQL connection
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}")

In [ ]:
from sqlalchemy import text
import pandas as pd

# SQL query to pull desired data for the most recent CostReport data for all Providers
query = """
        SELECT t1.prov_id, p.overall_rating, t1.loc_type, t1.ownership, t1.snf_num_beds, t1.total_assets 
        FROM CostReports t1
        JOIN (
                SELECT prov_id, MAX(fiscal_end) AS max_fiscal_end
                FROM CostReports
                GROUP BY prov_id
        ) t2 ON t1.prov_id = t2.prov_id AND t1.fiscal_end = t2.max_fiscal_end
        LEFT JOIN (
                SELECT DISTINCT prov_id, overall_rating
                FROM ProviderInfo
       ) p ON t1.prov_id = p.prov_id;
        """

# Perform the query using the engine and load the results to a dataframe
with engine.connect() as connection:
    result = connection.execute(text(query))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())

    engine.dispose()

In [ ]:
from sklearn.preprocessing import QuantileTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Preprocessing
categorical_cols = ['loc_type', 'ownership']
quantitative_cols = ['snf_num_beds', 'total_assets', 'overall_rating']

# Define imputers for different data types
imputer_quantitative = SimpleImputer(strategy= 'median')  # Impute with median for quantitative data
imputer_categorical = SimpleImputer(strategy= 'most_frequent')  # Impute with most frequent value for categorical data

# Define preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', Pipeline([
            ('imputer_categorical', imputer_categorical),
            ('encoder', OneHotEncoder())
        ]), categorical_cols),
        ('quant', Pipeline([
            ('imputer_quantitative', imputer_quantitative),
            ('scaler', QuantileTransformer())
        ]), quantitative_cols)
    ])

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.manifold import TSNE
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import FunctionTransformer

# Define a function to pass through TSNE transformation
tsne_transformer = FunctionTransformer(func=lambda X: TSNE(n_components=2, init = 'random').fit_transform(X))

# Clustering pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('tsne', tsne_transformer),  # for dimensionality reduction using t-SNE
    ('clustering', AgglomerativeClustering())
])

# Fit the pipeline
pipeline.fit(df.drop(columns=['prov_id']))

# Transform data and predict clusters
transformed_data = pipeline.named_steps['tsne'].transform(pipeline.named_steps['preprocessor'].transform(df.drop(columns=['prov_id'])))
clusters = pipeline.named_steps['clustering'].labels_

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Calculate silhouette score
silhouette = silhouette_score(transformed_data, clusters)

# Calculate Davies-Bouldin index
davies_bouldin = davies_bouldin_score(transformed_data, clusters)

# Add cluster labels to the DataFrame
df['cluster'] = clusters

# Print clustering metrics
print(f"Silhouette Score: {silhouette}")
print(f"Davies-Bouldin Index: {davies_bouldin}")

In [ ]:
import matplotlib.pyplot as plt

# Scatter plot for clustering visualization
plt.figure(figsize=(8, 6))
for cluster in df['cluster'].unique():
    plt.scatter(transformed_data[df['cluster'] == cluster, 0], transformed_data[df['cluster'] == cluster, 1], s= 5)
plt.show()